In [4]:
import pandas as pd

set_i = pd.read_csv(r"C:\Users\Albin Varun\Downloads\Set-II.csv")
set_ii = pd.read_csv(r"C:\Users\Albin Varun\Downloads\Set-II.csv")

# Merge the datasets
data = pd.concat([set_i, set_ii], ignore_index=True)

# View the first few rows of the data
data.head()


,Tweets,label
0,The Wave of #ChatGPT is incredible!! 🙄😳\n#AI\n...,Positive
1,My first dig at #ChatGPT and I am wondering if...,Neutral
2,does it feel rude to anyone else to press [Sto...,Neutral
3,#ChatGPT assisting me with #php coding,Positive
4,"When you ask #ChatGPT: \n""How do collaborative...",Positive


In [7]:
pip install tensorflow


     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 2.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/390.3 MB 2.0 MB/s eta 0:03:11
   ---------------------------------------- 0.1/390.3 MB 1.3 MB/s eta 0:04:56
   ---------------------------------------- 0.1/390.3 MB 1.2 MB/s eta 0:05:26
   ---------------------------------------- 0.2/390.3 MB 1.4 MB/s eta 0:04:41
   ---------------------------------------- 0.2/390.3 MB 1.3 MB/s eta 0:05:10
   ---------------------------------------- 0.3/390.3 MB 1.3 MB/s eta 0:05:08
   ---------------------------------------- 0.4/390.3 MB 1.2 MB/s eta 0:05:15
   ---------------------------------------- 0.4/390.3 MB 1.2 MB/s eta 0:05:13
   ---------------------------------------- 0.5/390.3 MB 1.2 MB/s eta 0:05:32
   ---------------------------------------- 0.5/390.3 MB 1.3 MB/s eta 0:05


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import re

# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    return text

# Apply the cleaning function
data['cleaned_text'] = data['Tweets'].apply(clean_text)

# Tokenize the text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])

# Pad sequences
X = pad_sequences(sequences, maxlen=100)

# Prepare the labels (assuming the sentiment is in the 'sentiment' column)
#y = data['label'].values  # Assuming 1 for positive and 0 for negative
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['label'])


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# One-hot encode the labels
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

# Define the RNN model
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    SimpleRNN(128, return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_one_hot, epochs=5, batch_size=32, validation_data=(X_test, y_test_one_hot))


Epoch 1/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.5824 - loss: 0.9846 - val_accuracy: 0.5762 - val_loss: 0.9370
Epoch 2/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6675 - loss: 0.8022 - val_accuracy: 0.5823 - val_loss: 0.7703
Epoch 3/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7709 - loss: 0.5273 - val_accuracy: 0.8476 - val_loss: 0.4222
Epoch 4/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9630 - loss: 0.1177 - val_accuracy: 0.8963 - val_loss: 0.3034
Epoch 5/5
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9954 - loss: 0.0271 - val_accuracy: 0.9085 - val_loss: 0.2961


In [21]:
# Save the model to disk
model.save('sentiment_rnn_model.h5')
